#Building a song recommender


#Fire up GraphLab Create

In [1]:
import graphlab as gl
gl.canvas.set_target('ipynb')

In [2]:
song_data = gl.SFrame('data/song_data.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to karthi.aru@gmail.com and will expire on May 31, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1480986746.log


In [3]:
kayne = song_data[song_data['artist'] == 'Kanye West']

In [4]:
len(kayne['user_id'].unique())

2522

In [5]:
foo = song_data[song_data['artist'] == 'Foo Fighters']

In [6]:
len(foo['user_id'].unique())

2055

In [7]:
taylor = song_data[song_data['artist'] == 'Taylor Swift']

In [8]:
len(taylor['user_id'].unique())

3246

In [9]:
lady = song_data[song_data['artist'] == 'Lady GaGa']

In [10]:
len(lady['user_id'].unique())

2928

In [11]:
popArt = song_data.groupby(key_columns='artist', operations={'total_count': gl.aggregate.SUM('listen_count')})

In [12]:
popArt.sort('total_count',ascending=False)

artist,total_count
Kings Of Leon,43218
Dwight Yoakam,40619
Björk,38889
Coldplay,35362
Florence + The Machine,33387
Justin Bieber,29715
Alliance Ethnik,26689
OneRepublic,25754
Train,25402
The Black Keys,22184


In [13]:
popArt.sort('total_count',ascending=True)

artist,total_count
William Tabbert,14
Reel Feelings,24
Beyoncé feat. Bun B andSlim Thug ...,26
Diplo,30
Boggle Karaoke,30
harvey summers,31
Nâdiya,36
Kanye West / Talib Kweli/ Q-Tip / Common / ...,38
Aneta Langerova,38
Jody Bernal,38


In [14]:
popSong = song_data.groupby(key_columns='song', operations={'total_count': gl.aggregate.SUM('listen_count')})

In [15]:
popSong.sort('total_count',ascending=False)

song,total_count
You're The One - DwightYoakam ...,40619
Undo - Björk,36059
Revelry - Kings Of Leon,30391
Horn Concerto No. 4 in Eflat K495: II. Romance ...,21953
Sehr kosmisch - Harmonia,21646
Dog Days Are Over (RadioEdit) - Florence + The ...,15889
Secrets - OneRepublic,14149
Ain't Misbehavin - SamCooke ...,14029
Invalid - Tub Ring,13701
Catch You Baby (StevePitron & Max Sanna Radio ...,12506


In [16]:
popSong.sort('total_count',ascending=True)

song,total_count
Hubcap - Sleater-kinney,12
Younger Than Springtime -William Tabbert ...,14
Accidntel Deth (AlbumVersion) - Rilo Kiley ...,16
Jumpers (Album) -Sleater-kinney ...,17
Come Back Margaret -Camera Obscura ...,18
Marching Theme - NeutralMilk Hotel ...,18
Music Now - FrightenedRabbit ...,19
No Fit State - Hot Chip,19
Footloose And Fancy Free- Camera Obscura ...,20
Tchaparian - Hot Chip,20


##Simple popularity-based recommender

In [17]:
train_data,test_data = song_data.random_split(.8,seed=0)

In [18]:
subset_test_users = test_data['user_id'].unique()[0:10000]

In [19]:
personalized_model = gl.item_similarity_recommender.create(train_data,
                                                                user_id='user_id',
                                                                item_id='song')

Recsys training: model = item_similarity

Warning: Ignoring columns song_id, listen_count, title, artist;

To use one of these as a target column, set target = <column_name>

and use a method that allows the use of a target.

Preparing data set.

...

| 2.26s                               | 100              | 9952            |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 3.39523s

In [20]:
recon = personalized_model.recommend(subset_test_users,k=1)

recommendations finished on 1000/10000 queries. users per second: 8517.74

recommendations finished on 2000/10000 queries. users per second: 10855.2

recommendations finished on 3000/10000 queries. users per second: 12510.6

recommendations finished on 4000/10000 queries. users per second: 13567.3

recommendations finished on 5000/10000 queries. users per second: 14396

recommendations finished on 6000/10000 queries. users per second: 15006.1

recommendations finished on 7000/10000 queries. users per second: 15389.5

recommendations finished on 8000/10000 queries. users per second: 15675.9

recommendations finished on 9000/10000 queries. users per second: 15987.5

recommendations finished on 10000/10000 queries. users per second: 16100.7

In [21]:
reconSong = recon.groupby(key_columns='song', operations={'count': gl.aggregate.COUNT()})

In [22]:
reconSong.sort('count',ascending=False)

song,count
Undo - Björk,438
Secrets - OneRepublic,385
Revelry - Kings Of Leon,224
You're The One - DwightYoakam ...,164
Fireflies - CharttraxxKaraoke ...,120
Horn Concerto No. 4 in Eflat K495: II. Romance ...,95
Sehr kosmisch - Harmonia,95
Hey_ Soul Sister - Train,91
OMG - Usher featuringwill.i.am ...,63
Dog Days Are Over (RadioEdit) - Florence + The ...,47


In [23]:
reconSong.sort('count',ascending=True)

song,count
My Perfect Cousin - TheUndertones ...,1
Vitamin - Incubus,1
One Last Kiss (LPVersion) - Madina Lake ...,1
Spanish Caravan [NewStereo Mix] (Advanced ...,1
Cerdo - Molotov,1
Chapter Four - AvengedSevenfold ...,1
Bed - J. Holiday,1
Give A Little Bit -Supertramp ...,1
Song For You - AlexiMurdoch ...,1
Hate It Here - Wilco,1
